# Data Science Assignment

In [ ]:
# 1) Environment setup
try:
    import gdown
except Exception:
    !pip install --quiet gdown
    import gdown

import pandas as pd, numpy as np, matplotlib.pyplot as plt, os, re
print('Environment ready.')


Environment ready.


In [ ]:
# 2) Download datasets (use the same IDs as before or upload manually)
btc_sentiment_id = "1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf"
trader_data_id     = "1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs"

out_dir = "/content/ds_data"
os.makedirs(out_dir, exist_ok=True)

def download_drive(id, out_name):
    url = f"https://drive.google.com/uc?id={id}"
    out_path = os.path.join(out_dir, out_name)
    try:
        import gdown
        gdown.download(url, out_path, quiet=False)
    except Exception as e:
        print('gdown failed (likely permissions):', e)
    return out_path

btc_path = download_drive(btc_sentiment_id, "fear_greed.csv")
traders_path = download_drive(trader_data_id, "traders.csv")

# If downloads fail, upload files manually to Colab root and update these variables:
btc_path = btc_path if os.path.exists(btc_path) else '/content/fear_greed.csv'
traders_path = traders_path if os.path.exists(traders_path) else '/content/traders.csv'

print('Using paths:')
print('  Sentiment:', btc_path)
print('  Traders  :', traders_path)


Downloading...
From: https://drive.google.com/uc?id=1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf
To: /content/ds_data/fear_greed.csv
100%|██████████| 90.8k/90.8k [00:00<00:00, 62.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs
To: /content/ds_data/traders.csv
100%|██████████| 47.5M/47.5M [00:00<00:00, 144MB/s]

Using paths:
  Sentiment: /content/ds_data/fear_greed.csv
  Traders  : /content/ds_data/traders.csv


In [ ]:
# 3) Load datasets robustly
def safe_read_csv(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f'File not found: {path}\nPlease upload the file to Colab and re-run this cell.')
    return pd.read_csv(path, low_memory=False)

df_sent = safe_read_csv(btc_path)
df_sent.columns = df_sent.columns.str.strip()
print('\nSentiment columns:', df_sent.columns.tolist())
display(df_sent.head())

df_tr = safe_read_csv(traders_path)
df_tr.columns = df_tr.columns.str.strip()
print('\nTrader columns:', df_tr.columns.tolist())
display(df_tr.head())



Sentiment columns: ['timestamp', 'value', 'classification', 'date']


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05



Trader columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [ ]:
# 4) Robust column detection helpers and cleaning

def find_col(df, candidates):
    """Return the first column in df whose name matches any candidate (fuzzy, case-insensitive)."""
    cols = df.columns.tolist()
    lc = [c.lower() for c in cols]
    # exact normalized match (strip non-alnum)
    def norm(s): return re.sub(r'[^a-z0-9]', '', s.lower())
    for cand in candidates:
        nc = norm(cand)
        for i,c in enumerate(cols):
            if norm(c) == nc:
                return cols[i]
    # substring match
    for cand in candidates:
        for i,c in enumerate(cols):
            if cand.lower() in c.lower():
                return cols[i]
    return None

# Determine date column for traders
date_col = find_col(df_tr, ['date','time','timestamp','trade_time','created_at'])
if date_col is None:
    # fallback: first column
    date_col = df_tr.columns[0]
    print('Warning: could not find an explicit date/time column; using', date_col)

# create parsed time_parsed and date_only
df_tr['time_parsed'] = pd.to_datetime(df_tr[date_col], errors='coerce')
df_tr['date_only'] = pd.to_datetime(df_tr['time_parsed']).dt.date

# detect size/volume column
size_col = find_col(df_tr, ['size','qty','quantity','volume','amount'])
if size_col is None:
    df_tr['size'] = np.nan
    size_col = 'size'
    print('Note: size/volume column not found — created synthetic "size" with NaN values.')
else:
    print('Mapped size column ->', size_col)

# detect closed PnL column
pnl_col = find_col(df_tr, ['closedpnl','closed_pnl','closed pnl','pnl','profit','pl','realisedpnl','realized_pnl'])
if pnl_col is None:
    df_tr['closedPnL'] = 0.0
    pnl_col = 'closedPnL'
    print('Note: closed PnL column not found — created synthetic "closedPnL" with zeros.')
else:
    print('Mapped PnL column ->', pnl_col)

# detect leverage column
lev_col = find_col(df_tr, ['leverage','lev','levarage'])
if lev_col is None:
    df_tr['leverage'] = np.nan
    lev_col = 'leverage'
    print('Note: leverage column not found — created synthetic "leverage" with NaN.')
else:
    print('Mapped leverage column ->', lev_col)

# convert these columns to numeric where possible
for c in [size_col, pnl_col, lev_col]:
    df_tr[c] = pd.to_numeric(df_tr[c], errors='coerce')

print('\nFinal trader columns used: size ->', size_col, ', pnl ->', pnl_col, ', leverage ->', lev_col)
print('Number of trader rows before dropping missing dates:', len(df_tr))
df_tr = df_tr.dropna(subset=['date_only'])
print('Number of trader rows after dropping missing dates:', len(df_tr))

# show sample
display(df_tr[[date_col,'time_parsed','date_only', size_col, pnl_col, lev_col]].head())


Mapped size column -> Size Tokens
Mapped PnL column -> Closed PnL
Note: leverage column not found — created synthetic "leverage" with NaN.

Final trader columns used: size -> Size Tokens , pnl -> Closed PnL , leverage -> leverage
Number of trader rows before dropping missing dates: 211224
Number of trader rows after dropping missing dates: 211224


,Timestamp,time_parsed,date_only,Size Tokens,Closed PnL,leverage
0,1.730000e+12,1970-01-01 00:28:50,1970-01-01,986.87,0.0,NaN
1,1.730000e+12,1970-01-01 00:28:50,1970-01-01,16.00,0.0,NaN
2,1.730000e+12,1970-01-01 00:28:50,1970-01-01,144.09,0.0,NaN
3,1.730000e+12,1970-01-01 00:28:50,1970-01-01,142.98,0.0,NaN
4,1.730000e+12,1970-01-01 00:28:50,1970-01-01,8.73,0.0,NaN


In [ ]:
# 5) Aggregation by date — safe and robust

def total_volume_fn(x):
    try:
        # x should be numeric after conversion above; if not, fallback to count
        return x.dropna().abs().sum()
    except Exception:
        return x.count()

agg = df_tr.groupby('date_only').agg(
    trade_count = (size_col, 'count'),
    total_volume = (size_col, total_volume_fn),
    avg_leverage = (lev_col, 'mean'),
    total_pnl = (pnl_col, 'sum'),
    avg_size = (size_col, 'mean')
).reset_index()

# Ensure date column becomes datetime (not python date)
agg['date_only'] = pd.to_datetime(agg['date_only'], errors='coerce')

# Prepare sentiment dataframe for merging
# Make sure df_sent has a date-only column
sent_date_col = find_col(df_sent, ['date','day','timestamp','date_only'])
if sent_date_col is None:
    sent_date_col = df_sent.columns[0]
    print('Warning: sentiment date column not detected; using', sent_date_col)

df_sent['Date_parsed'] = pd.to_datetime(df_sent[sent_date_col], errors='coerce')
df_sent['date_only'] = pd.to_datetime(df_sent['Date_parsed']).dt.date

# find a classification-like column in sentiment
sent_class_col = find_col(df_sent, ['classification','label','sentiment','fear_greed','value','score','fear'])
if sent_class_col is None:
    df_sent['Classification'] = 'Unknown'
    sent_class_col = 'Classification'
    print('Note: classification/label not found in sentiment — filled with "Unknown".')
else:
    df_sent['Classification'] = df_sent[sent_class_col].astype(str)
    print('Mapped sentiment classification ->', sent_class_col)

# normalize date types for merge: use python date
agg['date_only'] = agg['date_only'].dt.date
df_sent['date_only'] = pd.to_datetime(df_sent['date_only']).dt.date

merged = agg.merge(df_sent[['date_only','Classification']].drop_duplicates(), on='date_only', how='left')
print('\nMerged sample:')
display(merged.head())

# Save processed CSV
processed_csv = os.path.join('/content/ds_data', 'aggregated_merged.csv')
os.makedirs('/content/ds_data', exist_ok=True)
merged.to_csv(processed_csv, index=False)
print('Saved processed aggregation to', processed_csv)


Mapped sentiment classification -> classification

Merged sample:


,date_only,trade_count,total_volume,avg_leverage,total_pnl,avg_size,Classification
0,1970-01-01,211224,9.765656e+08,NaN,1.029696e+07,4623.364979,NaN


Saved processed aggregation to /content/ds_data/aggregated_merged.csv


In [ ]:
# 6) Visualizations
import matplotlib.dates as mdates, os, shutil, glob
out_plots = '/content/outputs'; os.makedirs(out_plots, exist_ok=True)

try:
    if 'merged' not in globals():
        raise NameError('merged is not defined — aggregation must have failed earlier.')
    # Plot total_pnl over time if exists
    if 'total_pnl' in merged.columns:
        fig, ax = plt.subplots(figsize=(10,4))
        ax.plot(pd.to_datetime(merged['date_only']), merged['total_pnl'], marker='o', linewidth=1)
        ax.set_title('Total PnL by Date')
        ax.set_xlabel('Date'); ax.set_ylabel('Total PnL')
        ax.xaxis.set_major_locator(mdates.AutoDateLocator()); fig.autofmt_xdate()
        pnl_png = os.path.join(out_plots, 'total_pnl_by_date.png')
        fig.savefig(pnl_png, bbox_inches='tight'); plt.close(fig)
        print('Saved', pnl_png)
    else:
        print('total_pnl column not present in merged — skipping PnL plot.')

    # Boxplot of avg_leverage grouped by sentiment classification
    if 'avg_leverage' in merged.columns:
        merged_box = merged.dropna(subset=['avg_leverage'])
        if not merged_box.empty and 'Classification' in merged_box.columns:
            groups = merged_box.groupby('Classification')['avg_leverage'].apply(list)
            if len(groups) > 0:
                fig, ax = plt.subplots(figsize=(8,4))
                ax.boxplot(groups.tolist(), labels=groups.index.tolist())
                ax.set_title('Average Leverage by Sentiment Classification')
                ax.set_ylabel('Avg Leverage')
                leverage_png = os.path.join(out_plots, 'leverage_by_sentiment.png')
                fig.savefig(leverage_png, bbox_inches='tight'); plt.close(fig)
                print('Saved', leverage_png)
        else:
            print('Not enough data for leverage boxplot.')
    else:
        print('avg_leverage not present — skipping leverage boxplot.')

except Exception as e:
    print('Plot error:', e)

# Copy outputs into a repo-like folder in /content for easy zipping
repo_root = '/content/ds_Gurdeep_Singh'; os.makedirs(repo_root, exist_ok=True)
csv_dest = os.path.join(repo_root, 'csv_files'); os.makedirs(csv_dest, exist_ok=True)
out_dest = os.path.join(repo_root, 'outputs'); os.makedirs(out_dest, exist_ok=True)
for f in glob.glob('/content/ds_data/*.csv'): shutil.copy(f, csv_dest)
for f in glob.glob('/content/outputs/*'): shutil.copy(f, out_dest)
print('Prepared /content/ds_Gurdeep_Singh with csv_files/ and outputs/')

Saved /content/outputs/total_pnl_by_date.png
Not enough data for leverage boxplot.
Prepared /content/ds_Gurdeep_Singh with csv_files/ and outputs/


----

End of fixed notebook. This notebook includes robust column detection and safe aggregation so the KeyError should no longer occur. If you still see missing columns, open the first data preview cell and paste the trader dataset's column list here so the mapping can be fine-tuned.